In [ ]:
from IPython.core.display import HTML

def css_styling():
    styles = open('./styles/custom.css', 'r').read()
    return HTML(styles)

css_styling()

# Introduction

This notebook is part of a technical test for Laing O'Rourke.

## Problem Statement
In this hypothetical situation, the stakeholders of an online retailer would like to know:
- What products are successful?
- What times of the day/week/year are successful?
- Can customers be segmented?
- Can customers be targeted more effectively?

## Why Solve This Problem
Advertising can drastically increase in costs if the targetting in question is not effective. For instance, if the incorrect targetting is used, many people will read and possibly click on your advert but will not purchase anything. This has meant that you have paid for the advertisement to be put in front of a customer, however, it hasn't achived its goal.

## Data Description

This is a transnational data set which contains all the transactions occurring between **01/12/2010** and **09/12/2011** for a UK-based and registered non-store online retail. The company mainly sells unique all-occasion gifts. Many customers of the company are wholesalers.

| Feature     | Description                                                                                                                                                 |
|-------------|-------------------------------------------------------------------------------------------------------------------------------------------------------------|
| InvoiceNo   | Invoice number. Nominal, a 6-digit integral number uniquely assigned to each transaction. If this code starts with letter 'c', it indicates a cancellation. |
| StockCode   | Product (item) code. Nominal, a 5-digit integral number uniquely assigned to each distinct product.                                                         |
| Description | Product (item) name. Nominal.                                                                                                                               |
| Quantity    | The quantities of each product (item) per transaction. Numeric.                                                                                             |
| InvoiceDate | Invice Date and time. Numeric, the day and time when each transaction was generated.                                                                        |
| UnitPrice   | Unit price. Numeric, Product price per unit in sterling.                                                                                                    |
| CustomerID  | Customer number. Nominal, a 5-digit integral number uniquely assigned to each customer.                                                                     |
| Country     | Country name. Nominal, the name of the country where each customer resides.                                                                                 |

# Imports

In [ ]:
# Linear Algebra
import numpy as np

# Data Processing
import pandas as pd

# Data Visualization
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.ticker import StrMethodFormatter
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.cm import get_cmap
import folium
from pywaffle import Waffle
import missingno as msno
from wordcloud import WordCloud
from PIL import Image

# Algorithms
from sklearn.cluster import KMeans
from sklearn.cluster import AgglomerativeClustering
from sklearn import metrics

# Stats
from scipy import stats

# Helper Functions
from HelperFunctions import *

# Set random seed for reproducibility
np.random.seed(0)
sns.set(style="whitegrid")  # Stylises graphs

In [ ]:
import sys
import matplotlib
import scipy
import sklearn

print("Python version: {}". format(sys.version))
print("pandas version: {}". format(pd.__version__))
print("matplotlib version: {}". format(matplotlib.__version__))
print("NumPy version: {}". format(np.__version__))
print("SciPy version: {}". format(scipy.__version__)) 
print("scikit-learn version: {}". format(sklearn.__version__))
print("Seaborn version: {}". format(sns.__version__))

<div class=warning>Please make sure that you are running Python ver. 3.6 or above and that the version print outs are equal to or above the requirements in <b>requirements.txt<\b>.</div>

# Reading in the Data

In [ ]:
df = pd.read_csv('./data/Online Retail.csv', encoding='ISO-8859-1')

In [ ]:
df.head()

In [ ]:
df.info()

<div class=info>
    Here we can see that everything has been read in fine. 33.1MB of memory has been used, meaning no dtype optimisation is needed.
    Some of the dtypes for the variables will need to be adjusted, notably, InvoiceDate.
</div>

# Data Cleaning
## Null Values

In [ ]:
round(df[df.columns].isnull().sum() * 100 / df.shape[0], 2)

In [ ]:
msno.matrix(df)

In [ ]:
msno.bar(df)

<div class=info>
    It mainly seems like Customer ID is missing values. This could cause issues. Description is also missing a small percentage of data. In most cases, this won't be an issue.
    There isn't a need to remove these rows, as other features witin the row will be useful.
<\div>

## Duplicates

In [ ]:
print(f'Duplicates: {df.duplicated().sum()}')

We can see here that there are `5268` duplicates. Let us remove these as the combination of `InvoiceNo` and `StockCode` should make a transaction unique. In the data spec the following is said.
> `InvoiceNo`: Invoice number. Nominal, a 6-digit integral number **uniquely assigned** to each transaction. If this code starts with letter 'c', it indicates a cancellation.

> `StockCode`: Product (item) code. Nominal, a 5-digit integral number **uniquely assigned** to each distinct product.

In [ ]:
df = df.drop_duplicates()

## InvoiceNo
`InvoiceNo` can start with the letter `c`, meaning a cancellation. We should remove this letter, so that we can change the data type of the column into an intger based one.

In [ ]:
df['Cancelled'] = df['InvoiceNo'].apply(lambda x: 'c' in x.lower())

Here we can see that the `Cancelled` column is `True` when there was a `c` in `InvoiceNo`.

In [ ]:
df[df['InvoiceNo'] == 'C544679']

Now let's remove all C from from each `InvoiceNo` and convert the values to integers.

In [ ]:
df['InvoiceNo'] = df['InvoiceNo'].str.replace('C|c', '')

In [ ]:
bad_values = check_coerce_problems(series=df['InvoiceNo'], dtype='numeric')

There are some values that begin with 'A'. There is nothing mentioned in the data description that talks about this. To be safe, these values should be removed.

In [ ]:
df = df[~df['InvoiceNo'].isin(bad_values)]
df['InvoiceNo'] = pd.to_numeric(df['InvoiceNo'], errors='raise')

In [ ]:
print(f'Duplicates: {df.duplicated().sum()}')

## InvoiceDate
Currently, `InvoiceDate` is in a string format. We would like this in a `datetime` format.

In [ ]:
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'], format='%d/%m/%Y %H:%M')

## Conclusion of Cleaning
More cleaning of this data isn't that necessary as there are not many missing values.
There could be some class inbalances but these can all be resolved easily here. For other more complex data sets, techniques like SMOTE could be used.

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.describe()

# Feature Engineering
It is important to extract extra features using data mining. These can suppliment analysis and machine learning performance.
## Datetime Engineering

In [ ]:
df['Month'] = df['InvoiceDate'].dt.month
df['Day'] = df['InvoiceDate'].dt.day
df['Hour'] = df['InvoiceDate'].dt.hour
df['Weekday'] = df['InvoiceDate'].dt.dayofweek

## Customer Frequency
Here we want to know how many invoices a customer has. This is different to the number of transactions a customer has.

In [ ]:
customer_freqs = df.groupby('CustomerID')['InvoiceNo'].nunique()
customer_freqs.name = 'Frequency'

## Transaction Total

In [ ]:
df['TotalPrice'] = df['Quantity'] * df['UnitPrice']

## Conclusion of Feature Engineering

In [ ]:
df.head()

# Exploratory Data Analysis
## Countries with Most Transations

In [ ]:
country_counts = df['Country'].value_counts().reset_index()
country_counts.columns = ['Country', 'Number of Invoices']
pd.DataFrame(country_counts.head(10))  # Turning into a dataframe for aesthetic reasons

<div class=info>The UK has by far the most invoices. It may be important to omit the UK from some analysis and visualisations.</div>

## Countries with Least Transations

In [ ]:
pd.DataFrame(country_counts.tail(10))

## Interactive Map of Counties with Highest Revenue and Quanitity of Products Sold
With the below map, you can select the overlay of data by clicking on the button in the top right of the map. It is highly recommended that you do so, as you will be able to view all the results.

In [ ]:
map_plotting = df.groupby(by='Country').sum().reset_index()
map_plotting = map_plotting[map_plotting['Country'] != 'United Kingdom']

geo_data = './data/countries.geojson'

m = folium.Map(location=[0, 0], zoom_start=2.5)

folium.Choropleth(
    geo_data=geo_data,
    name='Amount Spent',
    data=map_plotting.groupby(by='Country').sum().reset_index(),
    columns=['Country', 'TotalPrice'],
    key_on='feature.properties.ADMIN',
    fill_color='BuPu',
    fill_opacity=0.7,
    line_opacity=0.5,
    legend_name='Amount Spent'
).add_to(m)

folium.Choropleth(
    geo_data=geo_data,
    name='Quanitity',
    data=map_plotting.groupby(by='Country').sum().reset_index(),
    columns=['Country', 'Quantity'],
    key_on='feature.properties.ADMIN',
    fill_color='YlGn',
    fill_opacity=0.7,
    line_opacity=0.5,
    legend_name='Quantity',
    show=False
).add_to(m)

folium.LayerControl().add_to(m)

m

<div class=warning>The UK has been ommitted as to not skew the scales.</div>

## Time Series Analysis
### Transactions per Week

In [ ]:
holidays = {
    "Easter Monday": "2011-04-25",
    "Spring Bank Holiday": "2011-05-30",
    "Summer Bank Holiday": "2011-08-29",
    "New Years Day": "2011-01-01",
    "Christmas Day": "2010-12-25",
    "Mothering Sunday": "2011-04-03",
    "Father's Day": "2011-06-19"
}


holidays_df = pd.DataFrame.from_dict(holidays, orient='index').reset_index()
holidays_df.columns = ['Holiday Name', 'Date']
holidays_df['Date'] = pd.to_datetime(holidays_df['Date'], yearfirst=True)

In [ ]:
date_range = pd.date_range(
    start=df['InvoiceDate'].min(),
    end=df['InvoiceDate'].max(),
    freq='D'
)

plotting = df.set_index('InvoiceDate').groupby(pd.Grouper(freq='D')).count()

# Create a 7 day rolling average
plotting = plotting.rolling('7D').sum()

fig, ax = plt.subplots(figsize=(20, 10))
ax.plot_date(
    x=mdates.date2num(plotting.index), y=plotting['InvoiceNo'],
    fmt='b-'
)

# Plot holidays
for index, row in holidays_df.iterrows():
    ax.axvline(
        x=mdates.date2num(row['Date']),
        label=row['Holiday Name'],
        linestyle='--', c=next(ax._get_lines.prop_cycler)['color']
    )

ax.xaxis.set_major_locator(mdates.MonthLocator())
ax.get_yaxis().set_major_formatter(StrMethodFormatter('{x:,.0f}'))

ax.set_title('1 Week Rolling Average of Transactions with Time')
ax.set_xlabel('Time')
ax.set_ylabel('Number of Transactions per Week')
ax.legend()
plt.show()

<div class=info>
    To no suprise, Q4 has the most transactions. It is also intresting to see how there are no transactions during the Christmas/New Year's break.
    Furthermore, you can see spikes in transactions just before holidays and with subsequent dips during and after the holiday concludes.
</div>

### Revenue per Week

In [ ]:
date_range = pd.date_range(
    start=df['InvoiceDate'].min(),
    end=df['InvoiceDate'].max(),
    freq='D'
)

# Don't want to use cancelled ivnocies as they have neg. revenue
mask = ~df['Cancelled']

plotting = df[mask].set_index('InvoiceDate').groupby(pd.Grouper(freq='D')).sum()

# Create a 7 day rolling average
plotting = plotting.rolling('7D').sum()

fig, ax = plt.subplots(figsize=(20, 10))
ax.plot_date(
    x=mdates.date2num(plotting.index), y=plotting['TotalPrice'],
    fmt='-'
)

# Plot holidays
for index, row in holidays_df.iterrows():
    ax.axvline(
        x=mdates.date2num(row['Date']),
        label=row['Holiday Name'],
        linestyle='--', c=next(ax._get_lines.prop_cycler)['color']
    )

ax.xaxis.set_major_locator(mdates.MonthLocator())
ax.get_yaxis().set_major_formatter(StrMethodFormatter('{x:,.0f}'))

ax.set_title('Weekly Revenue with Time')
ax.set_xlabel('Time')
ax.set_ylabel('Weekly Revenue')
ax.legend()
plt.show()

<div class=info>
    Again, here you can see spikes in transactions just before holidays and with subsequent dips during and after the holiday concludes.
    These dips during the holidays are more apperent with this data.
</div>

### Successful Hours of the Day

In [ ]:
# SET DATA 
hour_counts = df['Hour'].value_counts()

hour_counts = pd.DataFrame(hour_counts.sort_index()).reset_index()
hour_counts.columns = ['Hour', 'Number of Transactions']

fig, ax = plt.subplots(figsize=(20, 10))
sns.barplot(x='Hour', y='Number of Transactions', data=hour_counts)

ax.set_title('Number of Transactions For Each Hour')
ax.set_xlabel('Hour')
ax.set_ylabel('Transactions')
plt.show()

In [ ]:
sns.set(style="whitegrid", rc={"axes.facecolor": (0, 0, 0, 0)})  # Stylises graphs

df_months = df.copy()  # Use copy to protect the orginal
df_months = df_months[df_months['Country'] == 'United Kingdom']
df_months['Month'] = df_months['InvoiceDate'].dt.strftime('%b')
month_order = [
    'Jan', 'Feb', 'Mar',
    'Apr', 'May', 'Jun',
    'Jul', 'Aug', 'Sep',
    'Oct', 'Nov', 'Dec'
]

# Initialize the FacetGrid object
g = sns.FacetGrid(
    df_months, row='Month', hue='Month',
    aspect=20, height=1,
    row_order=month_order
)

# Draw the densities in a few steps
g.map(
    sns.kdeplot, 'Hour',
    clip_on=False, shade=True,
    alpha=1, lw=1.5, bw=0.25
)
g.map(sns.kdeplot, 'Hour', clip_on=False, color='black', lw=2, bw=0.25)
g.map(plt.axhline, y=0, lw=2, clip_on=False)

# Define and use a simple function to label the plot in axes coordinates
def label(x, color, label):
    ax = plt.gca()
    ax.text(
        0, 0.05, label, fontweight="bold", color=color,
        ha="left", va="center", transform=ax.transAxes
    )

g.map(label, "Hour")

# Set the subplots to overlap
g.fig.subplots_adjust(hspace=-0.5)

# Remove axes details that don't play well with overlap
g.set_titles("")
g.set(yticks=[])
g.despine(bottom=True, left=True)

<div class=info>
    Whilst this visualisation looks nice, it doesn't quite capture anything of note. The purpose of this graph is to show how successful times of the day shift as the year goes on.
    This graph should be excluded from a report going to a stakeholder.
</div>

In [ ]:
sns.set(style="whitegrid")  # Stylises graphs

## Number of Transactions per Weekday

In [ ]:
# MAKE SURE MATPLOTLIB IS UPDATED TO THE LATEST VERSION FOR THIS PLOT OR IT WILL NOT WORK!!!!!

# SET DATA 
weekday_counts = df['Weekday'].value_counts().sort_index().values.tolist()
        
# CREATE BACKGROUND
days = [
    'Mon', 'Tues', 'Wed',
    'Thurs', 'Fri', 'Sun',
    ''
]

# Angle of each axis in the plot
angles = [(n / 6) * 2 * np.pi for n in range(7)]

subplot_kw = {
    'polar': True
}

fig, ax = plt.subplots(figsize=(15, 15), subplot_kw=subplot_kw)
ax.set_theta_offset(np.pi / 2)
ax.set_theta_direction(-1)
ax.set_rlabel_position(0)

plt.xticks(angles, days)
plt.yticks(color="grey", size=10)

# PLOT
weekday_counts = weekday_counts + [weekday_counts[0]]  # Properly loops the circle back

ax.plot(angles, weekday_counts, linewidth=1, linestyle='solid', label="Number of Transactions")
ax.fill(angles,  weekday_counts, 'orange', alpha=0.1)

plt.title("Counts of Transactions for Each Weekday")
plt.legend(loc='upper right', bbox_to_anchor=(0.1, 0.1))

plt.show()

<div class=info>The number of transactions do seem to fluctate throughout the week here. Friday and Sunday see a drop off. It would be intresting to investigate why this happens.</div>
<div class=warning>The data does not have any invoices on Saturdays. This is why is has been excluded.</div>

## Word Cloud
Here we will try to get a sense of what products are being sold.

In [ ]:
text = ' '.join(description for description in df['Description'].dropna())
bag_mask = np.array(Image.open('images/bag.png'))

world_cloud = WordCloud(
    width=1000, height=1000,
    background_color='white', mask=bag_mask,
    contour_width=3, contour_color='red'
).generate(text)

plt.figure(figsize=(20, 10))
plt.imshow(world_cloud, interpolation='bilinear')
plt.axis('off')
plt.tight_layout()
plt.show()

<div class=info>These transactions would indicate that this retail store sells a variety of things, backing up what is stated in the data spec that <i>many customers of the company are wholesalers</i>.</div>

### Returned Items
Some items are not marked as cancelled but have negative quanitity. This would indicate that these were returned. The intrest in looking at the descriptions here is *why were they sent back*?

In [ ]:
mask = ~df['Cancelled'] & (df['Quantity'] <= 0)
text = ' '.join(description for description in df[mask]['Description'].dropna())
mask = np.array(Image.open('images/fragile.png'))

world_cloud = WordCloud(
    width=1000, height=1000,
    background_color='white', mask=mask,
    contour_width=3, contour_color='red'
).generate(text)

plt.figure(figsize=(20, 10))
plt.imshow(world_cloud, interpolation='bilinear')
plt.axis('off')
plt.tight_layout()
plt.show()

<div class=info>These descriptions would indicate that these items have problems with them and highlights the potential to remove these bits of data as they could skew further analysis.</div>

In [ ]:
df['Returned'] = ~df['Cancelled'] & (df['Quantity'] <= 0)
print(f"Number of items returned: {df['Returned'].sum()}")

## Customer Frequency

In [ ]:
plot_customer_freqs = [value for value in customer_freqs.values if value <= 30]

plt.figure(figsize=(20, 10))
sns.countplot(
    plot_customer_freqs
)
plt.ylabel('Number of Customers')
plt.xlabel('Number of Invoices')
plt.title('Frequency of Customers')
plt.show()

<div class=info>A large amount of people have only 1 or 2 orders. The majority have under 5 orders.</div>

## Average Customer Spending

In [ ]:
mask = ~df['Cancelled'] & ~df['Returned']
prices = df[mask].groupby(by='InvoiceNo').sum()['TotalPrice']
prices = [price for price in prices if price < 1000 and price > 0]

plt.figure(figsize=(20, 10))
sns.distplot(
    prices, kde=False
)
plt.xlabel('Invoice Price')
plt.title('Distribution of Total Invoice Prices')
plt.show()

<div class=info>What is intresting here are the spikes in distributions. In the data spec it is mentioned <i>many customers of the company are wholesalers</i>. These spikes could be prices in which discounts are applied.</div>

## Most Sold Products

In [ ]:
mask = ~df['Cancelled'] & ~df['Returned']
product_quantities = df[mask].groupby(by=['StockCode']).sum()['Quantity'].sort_values(ascending=False)
product_quantities = pd.DataFrame(product_quantities).reset_index().head(20)
product_quantities['Description'] = product_quantities['StockCode'].apply(
    lambda x: df[df['StockCode'] == x]['Description'].values[0]
)

plt.figure(figsize=(20, 10))
sns.barplot(
    x='StockCode', y='Quantity',
    data=product_quantities
)
plt.ylabel('Quantity Sold')
plt.xlabel('Stock Code')
plt.title('Top Selling Products')
plt.show()

In [ ]:
product_quantities.head(10)

## Most Cancelled and Returned Products

In [ ]:
mask = df['Cancelled'] | df['Returned']
product_quantities = df[mask]['StockCode'].value_counts().sort_values(ascending=False)
product_quantities = pd.DataFrame(product_quantities).reset_index().head(20)

plt.figure(figsize=(20, 10))
sns.barplot(
    x='index', y='StockCode',
    data=product_quantities
)
plt.xlabel('Stock Code')
plt.title('Most Cancelled and Returned Products')
plt.show()

# Analysing Customer Value

In order to analysise and quantify customer value, the approach taken is **RFM**.

- **R**ecency - Customers that have purchased more recently are more likely to purchase again.
- **F**requency - Customers that have made purchases are more likely to purchase again.
- **M**onetary Value - Customers that have spent more are more likely to purchase again.

Recency is the most important behaviour in identifying customers that will response well to advertising. Frequency is the second most important, with monetary value being the third most.
In this analysis each behaviour will be ranked between 1 and 10. Likely, there is a better scale, for instance, [scikit-learn's Standard Scaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html). For interpretability for the stakeholder this hasn't been used, at the potential cost of machine learning performance.

## Constructing RFM Table

### Monetary Value
Here we want to find the maximum invoice (not transaction) that a customer has issued. This is to then be tacked onto the previously generated customer freqeuncy table.

In [ ]:
keep_cols = ['CustomerID', 'TotalPrice', 'Frequency']
mask = ~df['Cancelled'] & ~df['Returned']

# The long statement first groups on Customer and their Invoice to get their total for an invoice
# It then groups on customer to find each customer's max spend on an invoice
customers = pd.merge(
    df[mask].groupby(by=['CustomerID', 'InvoiceNo']).sum().reset_index().groupby(by='CustomerID').max().reset_index(),
    customer_freqs,
    on='CustomerID', how='inner'
)[keep_cols]

customers.head()

### Recency
Next, we need to find out how recently, from the start date of the data set, was each invoice issued.

In [ ]:
df['Recency'] = (max(df['InvoiceDate']) - df['InvoiceDate']).dt.days
keep_cols = ['CustomerID', 'TotalPrice', 'Recency']

customers = pd.merge(
    df[mask].groupby(by='CustomerID').min()['Recency'].reset_index(),
    customers,
    on='CustomerID', how='left'
)
customers.columns = ['ID', 'Recency', 'Monetary Value', 'Frequency']

customers.head()

## Outliers
Removing outliers here is extremely important. This is because when asigning a 1 - 10 score for each behaviour, we don't want the distribution of scores to be heavily skewed to one side. Outliers can also amplify a strong correlation or make an otherwise strong correlation appear weak.

Using z-scores could be effective here, however, it is likely that the data is nonparametric. If the data is nonparametric, Dbscan and Isolation Forests can be good solutions.
Using interquatile range is a good rule of thumb and is easier to explain to a stakeholder. This could impact machine learning performance but doesn't take too much time to implement.
With more time, class inbalance correcting techniques and the aforementioned outlier removal methods could be implemented.

In [ ]:
outlier_cols = [
    'Monetary Value', 'Frequency'
]

outliers_df = pd.DataFrame(columns=customers.columns)

for col in outlier_cols:
    stat = customers[col].describe()
    print(stat)
    IQR = stat['75%'] - stat['25%']
    upper = stat['75%'] + 1.5 * IQR
    lower = stat['25%'] - 1.5 * IQR
    
    outliers = customers[(customers[col] > upper) | (customers[col] < lower)]

    if not outliers.empty:
        print(f'\nOutlier found in: {col}')
        outliers_df = pd.concat([outliers_df, outliers])
    else:
        print(f'\nNo outlier found in: {col}')

    print(f'\nSuspected Outliers Lower Bound: {lower}')
    print(f'Suspected Outliers Upper Bound: {upper}\n\n')

print(f'Number of outlier rows: {len(outliers_df)}')

del outliers

In [ ]:
customers = customers[~customers['ID'].isin(outliers_df['ID'].values)]

### RFM Scaling
Here the RFM values are scaled between 1 and 10. With more time, sklearn's standard scaler amongst other methods could be implemented here for better machine learning performance.

In [ ]:
customers['Recency'] = np.interp(customers['Recency'], (customers['Recency'].min(), customers['Recency'].max()), (10, 0))
customers['Frequency'] = np.interp(customers['Frequency'], (customers['Frequency'].min(), customers['Frequency'].max()), (0, 10))
customers['Monetary Value'] = np.interp(customers['Monetary Value'], (customers['Monetary Value'].min(), customers['Monetary Value'].max()), (0, 10))

In [ ]:
customers.head()

In [ ]:
plt.figure(figsize=(20, 10))
sns.distplot(
    customers['Recency'], kde=False, bins=10
)
plt.xlabel('Recency')
plt.title('Distribution of Customer Recencies')
plt.show()

In [ ]:
plt.figure(figsize=(20, 10))
sns.distplot(
    customers['Frequency'], kde=False, bins=10
)
plt.xlabel('Frequency')
plt.title('Distribution of Customer Frequencies')
plt.show()

In [ ]:
plt.figure(figsize=(20, 10))
sns.distplot(
    customers['Monetary Value'], kde=False, bins=10
)
plt.xlabel('Monetary Value')
plt.title('Distribution of Customer Monetary Values')
plt.show()

<div class=info>All three values for RFM experience both negative and positive skews here. This should be kept in mind.</div>

# Model Building
## Model Explanations
![alt-text](https://bigdataldn.com/wp-content/uploads/2017/05/2-variable-clustering.jpg)

An unsupervised machine learning model will need to be implented. Unsupervised learning is used on data sets that do not have labelled responses to look for previously undetected patterns in a data set, where it is not feasible to be done by hand.
### K Means Clustering
K Means clustering is perhaps the most used unsupervised machine learning technique. First, ***k*** clusters are chosen. A random initial centre for each cluster is chosen. The next two steps are applied a number of times:
1. Assign each observation to its nearest centre
2. Update the centre of each cluster based off the centre of respective observation

![alt-text](https://miro.medium.com/max/700/1*oj1MBxiPfnyeQC3HnCcJiw.gif)

### Hierarchical Clustering
With this technique, initially each data point is made to be its own cluster. With each iteration, similar clusters marget with one and another until ***k*** clusters are formed. The following steps are computed:
1. Compute the proximity matrix
2. Let each data point be a cluster
3. Merge the two closest clusters and update the proximity matrix - repeat until one cluster remains

With this, any number ***k*** clusters can be seleted.

Here we will only be using *agglomerative* clustering. With more time, it could be worth seeing how the other form (*divisive*) of hierarchical clustering performs.

![alt-text](https://dashee87.github.io/images/hierarch.gif)

#### Average Linkage
![alt-text](https://www.saedsayad.com/images/Clustering_average.png)

The distance between each point in one cluster to every point in the other cluster is averaged. Average linking does well if there is noise between clusters but is bias towards globular clusters.

#### Single Linkage
![alt-text](https://www.saedsayad.com/images/Clustering_single.png)

The *shortest* distance between two points in each cluster. Single linkage can separate non-elliptical shapes, providing the gap between a pair of clusters is not small. However, it cannot separate clusters efficiently if there is noise between clusters.

#### Complete Linkage
![alt-text](https://www.saedsayad.com/images/Clustering_complete.png)

The *longest* distance between two points in each cluster. In a stark contrast to single linkage, complete linkage excels in separating clusters if there is noise between clusters. However, like average linkage, it tents to be biased towards globular clusters and tends to break large clusters.

#### Ward's Method
![alt-text](https://miro.medium.com/max/491/1*2AYd0CXANWsM8MLwmrJzYQ.jpeg)

Ward's method is the same as average linkage, except it calcuates the sum of the square distances. It shares the same pros and cons of average linkage too.

---

With more time, the effectiveness of ***Spectral Clustering*** and ***Gaussian mixture models*** could be tested.

## Hyperparamter Tuning
One upside of the hyperparamter tuning, is that supervised learning methods usually have many more hyperparameters to tune.

For example, here is the *elbow technique* to find an optimal number of clusters for our data. To find the number of clusters you simply pick the *elbow* of the curve. It is not the most objective method of finding the best model.

In [ ]:
inertias = []
n_clusters = range(2, 11)

for n in n_clusters:
    kmeans = KMeans(n_clusters=n, random_state=0)
    kmeans.fit(customers.drop(columns=['ID']))
    inertias.append(kmeans.inertia_)

plt.figure(figsize=(20, 10))
plt.plot(
    n_clusters, inertias
)
plt.xlabel('Number of Clusters')
plt.ylabel('Inertia')
plt.title('Distribution of Customer Monetary Values')
plt.show()

In order to evaluate each method, we will be using *silhouette scoring* and *calinski harabaz indexing*.

### Silhouette Scoring
A silhouette score describes how similar a datapoint is to other datapoints within its own cluster, relative to datapoints not in its cluster. This is computed for every data point.
This score is bounded between -1 and 1.

A score of 1 suggests each cluster is very tight and -1 suggests that each cluster is almost random.

### Calinski Harabasz Indexing
A Calinski-Harabasz index describes the ratio of the comparison of variance of a datapoint compared to points in its own clusters against the points in other clusters. A high index score is good here.

In [ ]:
scores = {'Cluster': [], 'N Clusters': [], 'Silhouette Score': [], 'Calinski Harabasz Score': []}
names = ['kmeans', 'ward', 'complete', 'average', 'single']
n_clusters = range(2, 11)

for n in n_clusters:
    kmeans = KMeans(n_clusters=n, random_state=0).fit(customers.drop(columns=['ID']))
    ward = AgglomerativeClustering(n_clusters=n, linkage='ward').fit(customers.drop(columns=['ID']))
    complete = AgglomerativeClustering(n_clusters=n, linkage='complete').fit(customers.drop(columns=['ID']))
    average = AgglomerativeClustering(n_clusters=n, linkage='average').fit(customers.drop(columns=['ID']))
    single = AgglomerativeClustering(n_clusters=n, linkage='single').fit(customers.drop(columns=['ID']))
    
    clusters = [kmeans, ward, complete, average, single]
    
    for cluster, name in zip(clusters, names):
            silhouett_score = metrics.silhouette_score(customers.drop(columns=['ID']), cluster.labels_)
            calinski_harabasz_score = metrics.calinski_harabasz_score(customers.drop(columns=['ID']), cluster.labels_)

            scores['Cluster'].append(name)
            scores['N Clusters'].append(n)
            scores['Silhouette Score'].append(silhouett_score)
            scores['Calinski Harabasz Score'].append(calinski_harabasz_score)

In [ ]:
scores = pd.DataFrame(scores)
scores.head()

In [ ]:
plt.figure(figsize=(20, 10))

for cluster in names:
    filter_cluster = scores[scores['Cluster'] == cluster]['Silhouette Score']
    plt.plot(
        n_clusters, filter_cluster,
        label=cluster
    )

plt.xlabel('Number of Clusters')
plt.ylabel('Silhouette Score')
plt.title('Silhouette Scores of Models with Number of Clusters')
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(20, 10))

for cluster in names:
    filter_cluster = scores[scores['Cluster'] == cluster]['Calinski Harabasz Score']
    plt.plot(
        n_clusters, filter_cluster,
        label=cluster
    )

plt.xlabel('Number of Clusters')
plt.ylabel('Calinski Harabasz Score')
plt.title('Calinski Harabasz Scores of Models with Number of Clusters')
plt.legend()
plt.show()

In [ ]:
final_model = KMeans(n_clusters=3, random_state=0).fit(customers.drop(columns=['ID']))
customers['Cluster Label'] = final_model.labels_

The single linkage hierarchical model with 2 clusters offers the highest silhouette score, however, its Calinski Harabasz index is ***very*** low. For this reason, a kmeans clustering algorithm has been chosen with 3 clusters. It offers an acceptable silhouette and the highest Calinski Harabasz index.

## Results

In [ ]:
fig = plt.figure(figsize=(20, 20))
ax = Axes3D(fig)
label_hues = {
    0: 'tab:green',
    1: 'tab:red',
    2: 'tab:blue'
}

for cluster_label, centre in zip(customers['Cluster Label'].unique(), kmeans.cluster_centers_):
    # Plot customer data with cluster label colouring
    ax.scatter(
        customers[customers['Cluster Label'] == cluster_label]['Recency'],
        customers[customers['Cluster Label'] == cluster_label]['Monetary Value'],
        customers[customers['Cluster Label'] == cluster_label]['Frequency'],
        c=label_hues[cluster_label], label=f'Customer Group: {cluster_label}'
    )
    
    # Plot the centre of cluster
    ax.scatter(
        centre[0], centre[1], centre[2],
        c='black', marker='x',
        s=120, linewidths=3,
    ) 

ax.view_init(elev=23, azim=160)
ax.set_xlabel('Recency Score')
ax.set_ylabel('Monetary Value Score')
ax.set_zlabel('Frequency Score')
ax.set_title('KMeans Clustering of RFM Customer Data')
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(20, 10))
sns.boxplot(x='Cluster Label', y='Recency', palette=label_hues, data=customers)
plt.title('Box Plot of Customer Recency Values')
plt.show()

In [ ]:
plt.figure(figsize=(20, 10))
sns.boxplot(x='Cluster Label', y='Monetary Value', palette=label_hues, data=customers)
plt.title('Box Plot of Customer Monetary Values')
plt.show()

In [ ]:
plt.figure(figsize=(20, 10))
sns.boxplot(x='Cluster Label', y='Frequency', palette=label_hues, data=customers)
plt.title('Box Plot of Customer Frequency Values')
plt.show()

In [ ]:
model_groups = customers.groupby('Cluster Label').count()['ID']
model_groups = round(model_groups / model_groups.sum(), 2) * 100

legend = {
    'labels': [f"Customer Label: {k} ({v}%)" for k, v in model_groups.items()],
    'loc': 'lower left',
    'bbox_to_anchor': (0, -0.1),
    'ncol': len(customers),
    'framealpha': 0,
    'fontsize': 12
}
title = {
    'label': 'Percentage of Customer Groups',
    'loc': 'left',
    'fontdict': {'fontsize': 20}
}

fig = plt.figure(
    FigureClass=Waffle,
    rows=5, columns=10,
    values=model_groups,
    icons=['user', 'user', 'user'],
    font_size=20, icon_style='solid', icon_legend=True,
    figsize=(10, 10), legend=legend, title=title,
    colors=list(label_hues.values())
)

fig.show()

In [ ]:
def customer_country(ID):
    countries = df[df['CustomerID'] == ID]['Country'].unique()
    
    if len(countries) > 0:
        return countries[0]
    else:
        return None

customer_countries = {'ID': [], 'Country': []}

for ID in df['CustomerID'].unique():
    customer_countries['ID'].append(ID)
    customer_countries['Country'].append(customer_country(ID))
    
customer_countries = pd.DataFrame(customer_countries)

customers = pd.merge(
    customers,
    customer_countries,
    on='ID', how='left'
)

In [ ]:
customers['Total'] = customers['Recency'] + customers['Monetary Value'] + customers['Frequency']
customers.head()

In [ ]:
countries = {'Country': [], 0: [], 1: [], 2: [], 'Total': []}

for country in customers['Country'].unique():
    counts = customers[customers['Country'] == country]['Cluster Label'].value_counts()
    
    countries['Country'].append(country)
    countries['Total'].append(counts.sum())
    
    for cluster_label in [0, 1, 2]:
        if cluster_label in counts:
            countries[cluster_label].append(counts[cluster_label])
        else:
            countries[cluster_label].append(0)
countries = pd.DataFrame(countries)
countries.head()

In [ ]:
m = folium.Map(location=[0, 0], zoom_start=2.5)

folium.Choropleth(
    geo_data=geo_data,
    name='Average Customer Score',
    data=customers.groupby(by='Country').mean()['Total'].reset_index(),
    columns=['Country', 'Total'],
    key_on='feature.properties.ADMIN',
    fill_color='BuPu',
    fill_opacity=0.7,
    line_opacity=0.5,
    legend_name='Average Customer Score'
).add_to(m)

folium.Choropleth(
    geo_data=geo_data,
    name='Number of Customers',
    data=countries[countries['Country'] != 'United Kingdom'],
    columns=['Country', 'Total'],
    key_on='feature.properties.ADMIN',
    fill_color='BuPu',
    fill_opacity=0.7,
    line_opacity=0.5,
    legend_name='Number of Customers',
    show=False
).add_to(m)

folium.LayerControl().add_to(m)

m

In [ ]:
countries.head()

In [ ]:
plt.figure(figsize=(20, 10))
sns.barplot(
    x='Total', y='Country',
    data=countries[countries['Country'] != 'United Kingdom'].sort_values(ascending=False, by='Total')
)
plt.xlabel('Total Customers')
plt.show()

In [ ]:
c_plotting = countries[countries['Total'] >= 5].copy()

totals = [i + j + k for i, j, k in zip(c_plotting[0], c_plotting[1], c_plotting[2])]
zero_bars = [i / j * 100 for i, j in zip(c_plotting[0], totals)]
one_bars = [i / j * 100 for i, j in zip(c_plotting[1], totals)]
two_bars = [i / j * 100 for i, j in zip(c_plotting[2], totals)]

c_plotting['Total'] = totals
c_plotting[0] = zero_bars
c_plotting[1] = one_bars
c_plotting[2] = two_bars

c_plotting = c_plotting.sort_values(by=1, ascending=True)


plt.figure(figsize=(20, 10))

barWidth = 0.85

plt.bar(
    range(len(totals)), c_plotting[0],
    color='tab:green', edgecolor='white',
    width=barWidth, label='Customer Label: 0'
)

plt.bar(
    range(len(totals)), c_plotting[1], bottom=c_plotting[0],
    color='tab:red', edgecolor='white',
    width=barWidth, label='Customer Label: 1'
)

plt.bar(
    range(len(totals)), c_plotting[2],
    bottom=[i + j for i, j in zip(c_plotting[0], c_plotting[1])],
    color='tab:blue', edgecolor='white',
    width=barWidth, label='Customer Label: 2'
)
 

plt.xticks(range(len(totals)), c_plotting['Country'].unique())

plt.xlabel('Country')
plt.xticks(rotation=90)

plt.ylabel('Customer Label Share %')

plt.title('100% Stacked Bar Chart of Customer Labels with Each Country That Has >= 5 Customers')
plt.legend()

plt.show()


In [ ]:
mask = ~df['Cancelled'] & ~df['Returned'] & df['CustomerID'].isin(customers[customers['Cluster Label'] == 2]['ID'].values)
product_quantities = df[mask].groupby(by=['StockCode']).sum()['Quantity'].sort_values(ascending=False)
product_quantities = pd.DataFrame(product_quantities).reset_index().head(20)
product_quantities['Description'] = product_quantities['StockCode'].apply(
    lambda x: df[df['StockCode'] == x]['Description'].values[0]
)

plt.figure(figsize=(20, 10))
sns.barplot(
    x='StockCode', y='Quantity',
    data=product_quantities
)
plt.ylabel('Quantity Sold')
plt.xlabel('Stock Code')
plt.title('Top Selling Products Among Customers with a Customer Label of 2')
plt.show()

In [ ]:
product_quantities.head(10)

# Insights and Decisions

Cluster label `0`: These seem to be customers with that have recently bought products but don't have a high frequency and didn't buy anything expensive.

Cluster label `1`: These customers have low recency, frequency and monetary value. They have little value to the company.

Cluster label `2`: These customers are recent buyers with a wide range of freqencies and monetary values. These customers are of the most interest to the company.

It would make sense to focus efforts in targetting customers with a label of `2` as they boast high recency scores. Recency scoring being the most important behaviour. Along with this, customers with a label of `2` generally have greater frequency and monetary scores than that of customers with a label of `0` and `1`. A large majority of customers have a label of `0`. It could be argued that these also need to be targeted, in order to increase the frequency in which they purchase items.

The UK should be the focus of the majority of advertisement spend, however, France and Germany already have a comparitivley large amount of customers. Germany does have a smaller percentage of customers with a label of `1`. Advertising could be quite effective there.

Customers with a label of `2` seem to purchase differing items than that of the general customer base. The following would be effective items to be displayed within an advertisement towards these customers:
- WORLD WAR 2 GLIDERS ASSTD DESIGNS
- SMALL CHINESE STYLE SCISSOR
- ASSORTED COLOUR BIRD ORNAMENT
- JUMBO BAG RED RETROSPOT
- WHITE HANGING HEART T-LIGHT HOLDER

Advertising to these customers would be most effective midday, during the working week and closer towards the end of the year.

# Next Steps
The main next step would be to test other methods of:
- Class imbalances
- Outlier detection techniques
- Methods of scaling RFM values
- Unsupervised machine learning models
- Evaluative unsupervised machine learning methods

Testing all of these would undoubtably increase model performance.

Another key aspects of this process is the deployment, maintenance and updating of the model.
It would be key to work alongside a data architect to ensure that the flow of all three aspects are achieved.
New data would improve the performance of machine learning models, so that new versions of it can be deployed.